In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
import web_scraping as ws

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)

In [162]:
with open('combined_movie_df_6.pickle', 'rb') as read_file:
    movie_df = pickle.load(read_file)

In [262]:
with open('combined_movie_df_6.pickle', 'wb') as to_write:
    pickle.dump(movie_df, to_write)

In [106]:
# Put in data_cleaning module

def find_unique_genres(df):
    unique_genres = []
    genres_list_of_lists = df['genres'].value_counts().index.to_list()
    for genres_list in genres_list_of_lists:
        unique_genres += genres_list
#     unique_genres_set = set(unique_genres)
    return set(unique_genres)

# def clean_unique_genres_set(unique_genres_set):
#     cleaned_set = set()
#     for genre in unique_genres_set:
#         cleaned_set.add(genre.strip())
#     return cleaned_set

In [130]:
# Put in feature_engineering module

def create_genre_encodings(df, unique_genres):
    for genre in unique_genres:
        df['is_' + genre] = df.apply(lambda x: genre in x['genres'], axis=1).astype(int)

In [120]:
is_genre_cols = ['is_Action', 'is_Adventure',
                 'is_Animation', 'is_Biography', 'is_Comedy', 'is_Crime', 'is_Drama',
                 'is_Family', 'is_Fantasy', 'is_History', 'is_Horror', 'is_Music',
                 'is_Musical', 'is_Mystery', 'is_News', 'is_Romance', 'is_Sci-Fi',
                 'is_Sport', 'is_Thriller', 'is_War', 'is_Western']